# Исследование надёжности заёмщиков

## Оглавление:

1. [Изучаем входные данные]
2. [Предобработка данных]
- 2.1 Обработка пропусков и ошибок
    - Анализ пропусков
    - Устранение отрицательных значений
    - Устранение пропусков в данных
    - Устраняем ошибки в столбце 'children'
    - Устраняем ошибки в столбце 'gender'
    - Устраняем ошибки в столбце 'income_type'
- 2.2 Замена типа данных
- 2.3 Обработка дубликатов
- 2.4 Лемматизация данных
- 2.5 Категоризация данных
    - Категоризируем колонки "уровень дохода"
    - Категоризируем колонку трудовой стаж
    - Категоризируем колонку "возраст"
3. [Анализ данных] 
    - Анализируем наличие задолжностей по колличеству детей
    - Анализируем наличие задолжностей по семейному статусу
    - Анализируем наличие задолжностей по уровню дохода
    - Анализируем наличие задолжностей по уровню образования
    - Анализируем наличие задолжностей по цели получения кредита
    - Анализируем наличие задолжностей по гендорному признаку
    - Анализируем наличие задолжностей по типу занятости
    - Анализируем наличие задолжностей по трудовому стажу
    - Анализируем наличие задолжностей по категории возраста
4. Вывод

In [1]:
import pandas as pd
from pymystem3 import Mystem
m=Mystem()
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

# 1. Изучаем входные данные

In [2]:
# считываем датафрейм
try:
    df=pd.read_csv('C:\\\\Python\\\\ЯП\\\\проект_1\\\\data.csv')
except:
    df=pd.read_csv('/datasets/data.csv')

In [3]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Добавляем столбец заменяющий собой идентификатор строки. 
Изучаем датафрейм

In [4]:
df.shape

(21525, 12)

посмотрим на размеры датафрейма

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


посмотрим на типы данных: колонки days_employed=float64 и total_income=float64 

In [6]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

посмотрим на пропущенные значения

проанализируем наш датафрейм с помощью функции ".value_counts()

In [7]:
for i in df.columns:
    print('Анализ колонки: "{:}"'.format(i))
    print(df[i].value_counts())
    print('')
    print('')

Анализ колонки: "children"
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Анализ колонки: "days_employed"
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64


Анализ колонки: "dob_years"
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51

Выводы:

В "трудовом стаже" есть аномально большие значения - ошибки.

Возраст некоторых клиентов - 0 лет.

В датафрейме обнаружили пропущенные значения в колонках days_employed и total_income.Так же эти колонки имеют тип данных float, а должен быть int.

Некоторые значения в этих колонках принимают отрицательное значение, что тоже не верно, т.к. не могут быть отрицательными трудовой стаж и ежемесячный доход.

При осмотре датафрейма обнаружил дубликаты в колонке education. Дубликаты возникли из-за написания букв в разном регистре.

В колонке children неадекватное кол-во детей: "-1" и "20". Думаю что эта ошибка возникла из-за лишнего знака "минус"(в "-1") и лишнего 0(в "20").

В колонке gender странное значение XNA. Считаю что это просто ошибка.

В колонке purpose тоже много дубликатов написанных разным стилем (свадьба, сыграть свадьбу, и т.д.). Возникло это из-за того что существует возможность свободной записи (а не выбор из ограниченного числа вариантов). 

В колонке "income_type" слишком мало данных по значениям : "предприниматель", "безработный", "студент", "в декрете". Необходимо их удалить.

# 2. Предобработка данных

## 2.1  Обработка пропусков и ошибок

### Анализ пропусков

In [8]:
df_nan=df[df['days_employed'].isnull()==True]
df_nan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


создаем датафрейм с пропущенными значениями 'days_employed' и 'total_income'

In [9]:
for i in df_nan.columns:
    print('Анализ колонки "{:}"'.format(i))
    print(df_nan[i].value_counts())
    print('   ')
    print('   ')

Анализ колонки "children"
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64
   
   
Анализ колонки "days_employed"
Series([], Name: days_employed, dtype: int64)
   
   
Анализ колонки "dob_years"
34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
50    51
39    51
29    50
43    50
49    50
51    50
45    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
64    37
32    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
71     5
20     5
69     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64
   
   
Анализ колонки "education"
среднее                1408
высшее                  496
СРЕДНЕЕ                  67
Среднее                  65
неоконченное высшее      55
Высшее              

проанализируем наш датафрейм с помощью функции ".value_counts():

Большинство людей не указывающих трудовой стаж и ежемесячный доход:
1) не имеют детей

2) имеют среднее образование (3:1 = среднее:высшее)

3) состоят в браке

4) примерно 68% женщины, 32% мужчины

In [10]:
df_nan.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости


Расчитаем сколько у нас клиентов без детей от общей массы клиентов с NaN:

In [11]:
df_nan[df_nan['children']==0]['children'].count()/df_nan['children'].count()

0.6619135234590616

Посмотрим аналогичный показатель у датафрейма без пропущенных значений:

In [12]:
df[(df['days_employed'].isnull()==False)&(df['children']==0)]['children'].count()/df[(df['days_employed'].isnull()==False)]['children'].count()

0.6568136013642706

Вывод: Пропорция клиентов с 0 детей сохраняется.

Расчитаем сколько у нас клиентов старше 40 лет от общей массы клиентов с NaN:

In [13]:
df_nan[df_nan['dob_years']>40]['dob_years'].count()/df_nan['dob_years'].count()

0.5676172953081877

Посмотрим аналогичный показатель у датафрейма без пропущенных значений:

In [14]:
df[(df['days_employed'].isnull()==False)&(df['dob_years']>40)]['dob_years'].count()/df[(df['days_employed'].isnull()==False)]['dob_years'].count()

0.5541832463438582

Вывод: Пропорция клиентов старше 40 лет от общей массы клиентов сохраняется.

Расчитаем сколько у нас клиентов c средним образованием от общей массы клиентов с NaN:

In [15]:
df_nan[(df_nan['education_id']==1)]['education_id'].count()/df_nan['education_id'].value_counts().sum()

0.7083716651333947

Посмотрим аналогичный показатель у датафрейма без пропущенных значений:

In [16]:
df[(df['days_employed'].isnull()==False)&(df['education_id']==1)]['education_id'].count()/df[(df['days_employed'].isnull()==False)]['education_id'].value_counts().sum()

0.7076120097152602

 Вывод: Пропорция клиентов с средним образованием сохраняется.

Расчитаем сколько у нас клиентов мужчин/женщин от общей массы клиентов с NaN:

In [17]:
df_nan[(df_nan['gender']=='F')]['gender'].count()/df_nan['gender'].value_counts().sum()

0.6826126954921803

Посмотрим аналогичный показатель у датафрейма без пропущенных значений:

In [18]:
df[(df['days_employed'].isnull()==False)&(df['gender']=='F')]['gender'].count()/df[(df['days_employed'].isnull()==False)]['gender'].value_counts().sum()

0.6589840318329802

Вывод: Пропорция клиентов по гендорному признаку сохраняется.

Расчитаем сколько у нас клиентов пенсионеров от общей массы клиентов с NaN:

In [19]:
df_nan[(df_nan['income_type']=='пенсионер')]['income_type'].count()/df_nan['income_type'].value_counts().sum()

0.18997240110395583

Посмотрим аналогичный показатель у датафрейма без пропущенных значений:

In [20]:
df[(df['days_employed'].isnull()==False)&(df['income_type']=='пенсионер')]['income_type'].count()/df[(df['days_employed'].isnull()==False)]['income_type'].value_counts().sum()

0.1779236215182678

Вывод: Пропорция клиентов пенсионеров сохраняется.

В итоге по NaN: 

никаких аномалий в пропусках не нашлось (например, что пенсионеры "наошибались" больше или клиенты без детей). Поэтому считаем, что типы пропусков - случайные.

Однако, заметим что пропуски идут "парами" в колонках "трудовой стаж" и "ежемесячный доход". Возможно что-то не так с выгрузкой данных по этим параметрам. 

### Устранение отрицательных значений

In [21]:
df['days_employed'] = abs(df['days_employed'])
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [22]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Убедились что знака минуса нет

### Редактирование аномальных значений в таблице со стажем

In [23]:
df['days_employed'].max()

401755.40047533

401755:365 (дней в году) = 1100 лет трудового стажа. Многовато. Отсортируем нашу таблицу по принципу: не более 70 лет трудового стажа!(с запасом на "героев труда"). 365*70=25550

In [24]:
df.shape

(21525, 12)

In [25]:
df[df['days_employed']>=25550]['days_employed'].count()

3445

Посмотрели сколько у нас аномалий. Выкидывать их не стоит т.к. их слишком много (порядка 16%). 

### Исправим "трудовой стаж"

Сначала посмотрим какая категория людей у нас наошибалась:

In [26]:
df[df['days_employed']>=25550]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Пенсионеры

Найдем медианное значение ошибки

In [27]:
error_stag_day=df[df['days_employed']>=25550]['days_employed'].median()
error_stag_day

365213.30626573117

Посмотрим сколько это в годах:

In [28]:
error_stag_day/365

1000.5844007280306

1000 лет - многовато....



может забыли перевести из секунд в минуты (или из минут в часы):

In [29]:
error_stag_day/365/60

16.67640667880051

Получилось 16 лет - трудовой стаж - маловато для пенсионеров

может забыли поделить на 7 дней в неделю:

In [30]:
error_stag_day/365/7

142.94062867543295

143 года - слишком много

может забыли поделить на 30 дней в месяце:

In [31]:
error_stag_day/365/30

33.35281335760102

33 года - такой трудовой стаж возможен! Делаем предположение что выгружая данные забыли поделить на 30. 
Дальнейшие рассуждения: напишем функцию которая будет делить значения стажа большие 25550 дней на 30 и применим её к нашему датафрейму.

создаем копию df_1 для второго варианта решения

In [32]:
df_1=df.copy()

Создаем функцию которая будет делить значения больше 25550 дней на 30.

In [33]:
def funk_30(stolb):
    if stolb>=25550:
        return stolb/30
    else:
        return stolb

In [34]:
df['days_employed']=df['days_employed'].apply(funk_30)

Отредактировали столбец!

In [35]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,11342.202402,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,11464.580138,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Убедились, что больше больших значений не осталось

In [36]:
df[df['days_employed']>=25550]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [37]:
df.shape

(21525, 12)

### Устранение пропусков в данных

### Сперва устраним пропуски в столбце "days_employed".

Считаем, что сильнее всего на параметр - трудовой стаж влияет возраст клиента. Гендерные различия,по идее, тоже должны быть, т.к. женщины уходят на пенсию раньше мужчин.

Пропущенные значения разобьем на группы:

1) женщины возрастом до 23 лет включительно ( _1 )

2) мужчины возрастом до 23 лет включительно ( _2 )

3) женщины от 23 до 35 лет ( _3 )

4) мужчины от 23 до 35 лет ( _4 )

5) женщины от 35 до 65 лет ( _5 )

6) мужчины от 35 до 65 лет ( _6 )

7) женщины возрастом после 65 лет включительно ( _7 )

8) мужчины возрастом после 65 лет включительно ( _8 )

И в кажой группе заменим пропущенное значение на медианное из аналогичной группы.

Смотрим сколько у нас людей с нулевым возрастом - ошибок:

In [38]:
df[df['dob_years']==0]['dob_years'].count()

101

удаляем строки с нулевым возрастом

In [39]:
df=df.drop(df[df['dob_years']==0].index)

Проверяем

In [40]:
df[df['dob_years']==0]['dob_years'].count()

0

In [41]:
days_employed_median_1=df[(df['gender']=='F') & (df['dob_years']<=23)]['days_employed'].median()
days_employed_median_2=df[(df['gender']=='M') & (df['dob_years']<=23)]['days_employed'].median()

days_employed_median_3=df[(df['gender']=='F') & (df['dob_years']>23)& (df['dob_years']<=35)]['days_employed'].median()
days_employed_median_4=df[(df['gender']=='M') & (df['dob_years']>23)& (df['dob_years']<=35)]['days_employed'].median()

days_employed_median_5=df[(df['gender']=='F') & (df['dob_years']>35)& (df['dob_years']<65)]['days_employed'].median()
days_employed_median_6=df[(df['gender']=='M') & (df['dob_years']>35)& (df['dob_years']<65)]['days_employed'].median()

days_employed_median_7=df[(df['gender']=='F') & (df['dob_years']>=65)]['days_employed'].median()
days_employed_median_8=df[(df['gender']=='M') & (df['dob_years']>=65)]['days_employed'].median()

Вычислили медианные значения

In [42]:
[days_employed_median_1,days_employed_median_2,days_employed_median_3,days_employed_median_4,days_employed_median_5,days_employed_median_6,days_employed_median_7,days_employed_median_8]

[659.7022899427086,
 722.6599458448883,
 1366.9036196255645,
 1188.8265441788096,
 3395.981436021678,
 2158.8499867917585,
 12080.898852460557,
 11863.267377186121]

Посмотрим на них. Значения сильно различаются в зависимости от возраста.

Заменили значения days_employed соответственно каждой из групп

In [43]:
df.loc[(df.loc[:,'days_employed'].isnull()==True)&
       (df.loc[:,'gender']=='F')&
       (df.loc[:,'dob_years']<=23),'days_employed'] = df.loc[(df.loc[:,'days_employed'].isnull()==True)&
       (df.loc[:,'gender']=='F')&
       (df.loc[:,'dob_years']<=23),'days_employed'].fillna(days_employed_median_1)




df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']<=23),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']<=23),'days_employed'].fillna(days_employed_median_2)

df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='F')&(df.loc[:,'dob_years']>23)&(df.loc[:,'dob_years']<=35),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='F')&(df.loc[:,'dob_years']>23)&(df.loc[:,'dob_years']<=35),'days_employed'].fillna(days_employed_median_3)
df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']>23)&(df.loc[:,'dob_years']<=35),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']>23)&(df.loc[:,'dob_years']<=35),'days_employed'].fillna(days_employed_median_4)

df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='F')&(df.loc[:,'dob_years']>35)&(df.loc[:,'dob_years']<65),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='F')&(df.loc[:,'dob_years']>35)&(df.loc[:,'dob_years']<65),'days_employed'].fillna(days_employed_median_5)
df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']>35)&(df.loc[:,'dob_years']<65),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']>35)&(df.loc[:,'dob_years']<65),'days_employed'].fillna(days_employed_median_6)

df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='F')&(df.loc[:,'dob_years']>=65),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='F')&(df.loc[:,'dob_years']>=65),'days_employed'].fillna(days_employed_median_1)
df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']>=65),'days_employed']=df.loc[(df.loc[:,'days_employed'].isnull()==True)&(df.loc[:,'gender']=='M')&(df.loc[:,'dob_years']>=65),'days_employed'].fillna(days_employed_median_2)



Заполним пропуски в таблице. 

## Теперь устраним пропуски в столбце "total_income" (аналогичным образом):

Считаем, что сильнее всего на параметр ежемесячный доход влияет "тип занятости"(пенсионер/сотрудник/...) и уровень образования.

Пропущенные значения разобьем на группы:

1) сотрудник с высшим образованием ('education_id'= 0) ( _1 )

2) компаньон с высшим образованием ( _2 )

3) пенсионер с высшим образованием ( _3 )

4) госслужащий с высшим образованием ( _4 )

..........................................

16) госслужащий с начальнам образованием ( _16 )

И в кажой группе заменим пропущенное значение на медианное из аналогичной группы.
Обращаемся к столбцу 'education_id' так как столбец 'education' еще не обработан (содержит дубликаты).



In [44]:
total_income_median_1=df[(df['education_id']==0) & (df['income_type']=='сотрудник')]['total_income'].median()
total_income_median_2=df[(df['education_id']==0) & (df['income_type']=='компаньон')]['total_income'].median()
total_income_median_3=df[(df['education_id']==0) & (df['income_type']=='пенсионер')]['total_income'].median()
total_income_median_4=df[(df['education_id']==0) & (df['income_type']=='госслужащий')]['total_income'].median()

total_income_median_5=df[(df['education_id']==2) & (df['income_type']=='сотрудник')]['total_income'].median()
total_income_median_6=df[(df['education_id']==2) & (df['income_type']=='компаньон')]['total_income'].median()
total_income_median_7=df[(df['education_id']==2) & (df['income_type']=='пенсионер')]['total_income'].median()
total_income_median_8=df[(df['education_id']==2) & (df['income_type']=='госслужащий')]['total_income'].median()

total_income_median_9=df[(df['education_id']==1) & (df['income_type']=='сотрудник')]['total_income'].median()
total_income_median_10=df[(df['education_id']==1) & (df['income_type']=='компаньон')]['total_income'].median()
total_income_median_11=df[(df['education_id']==1) & (df['income_type']=='пенсионер')]['total_income'].median()
total_income_median_12=df[(df['education_id']==1) & (df['income_type']=='госслужащий')]['total_income'].median()

total_income_median_13=df[(df['education_id']==3) & (df['income_type']=='сотрудник')]['total_income'].median()
total_income_median_14=df[(df['education_id']==3) & (df['income_type']=='компаньон')]['total_income'].median()
total_income_median_15=df[(df['education_id']==3) & (df['income_type']=='пенсионер')]['total_income'].median()
total_income_median_16=df[(df['education_id']==3) & (df['income_type']=='госслужащий')]['total_income'].median()



In [45]:
[total_income_median_1,
 total_income_median_2,
 total_income_median_3,
 total_income_median_4,
 total_income_median_5,
 total_income_median_6,
 total_income_median_7,
 total_income_median_8,
 total_income_median_9,
 total_income_median_10,
 total_income_median_11,
 total_income_median_12,
 total_income_median_13,
 total_income_median_14,
 total_income_median_15,
 total_income_median_16]

[165845.77847553013,
 201660.7398627252,
 144240.76861073746,
 172511.10701550177,
 151308.9378458309,
 179867.15288993614,
 120136.89635252525,
 160592.34530334338,
 136585.18658633163,
 159039.1889087703,
 114890.02634200183,
 136954.03261170606,
 125994.91060275196,
 136798.9051432623,
 102598.65316356516,
 148339.29082488286]

Посмотрим на них: различия есть.

In [46]:
df.loc[(df.loc[:,'total_income'].isnull()==True)&
       (df.loc[:,'education_id']==0)&
       (df.loc[:,'income_type']=='сотрудник'),
       'total_income'] = df.loc[(df.loc[:,'total_income'].isnull()==True)&
       (df.loc[:,'education_id']==0)&
       (df.loc[:,'income_type']=='сотрудник'),
       'total_income'].fillna(total_income_median_1)


df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==0)&(df.loc[:,'income_type']=='компаньон'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==0)&(df.loc[:,'income_type']=='компаньон'),'total_income'].fillna(total_income_median_2)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==0)&(df.loc[:,'income_type']=='пенсионер'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==0)&(df.loc[:,'income_type']=='пенсионер'),'total_income'].fillna(total_income_median_3)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==0)&(df.loc[:,'income_type']=='госслужащий'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==0)&(df.loc[:,'income_type']=='госслужащий'),'total_income'].fillna(total_income_median_4)

df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='сотрудник'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='сотрудник'),'total_income'].fillna(total_income_median_5)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='компаньон'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='компаньон'),'total_income'].fillna(total_income_median_6)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='пенсионер'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='пенсионер'),'total_income'].fillna(total_income_median_7)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='госслужащий'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==2)&(df.loc[:,'income_type']=='госслужащий'),'total_income'].fillna(total_income_median_8)

df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='сотрудник'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='сотрудник'),'total_income'].fillna(total_income_median_9)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='компаньон'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='компаньон'),'total_income'].fillna(total_income_median_10)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='пенсионер'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='пенсионер'),'total_income'].fillna(total_income_median_11)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='госслужащий'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==1)&(df.loc[:,'income_type']=='госслужащий'),'total_income'].fillna(total_income_median_12)

df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='сотрудник'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='сотрудник'),'total_income'].fillna(total_income_median_13)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='компаньон'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='компаньон'),'total_income'].fillna(total_income_median_14)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='пенсионер'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='пенсионер'),'total_income'].fillna(total_income_median_15)
df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='госслужащий'),'total_income']=df.loc[(df.loc[:,'total_income'].isnull()==True)&(df.loc[:,'education_id']==3)&(df.loc[:,'income_type']=='госслужащий'),'total_income'].fillna(total_income_median_16)


Заполнили пустые колонки в соответствии с категориями

In [47]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,11342.202402,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [48]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
dtype: int64

Смотрим на оставшееся кол-во пропущенных значений: 1 шт. В дальнейшем мы его удалим. (это "предприниматель")

In [49]:
df.shape

(21424, 12)

проверяем размер

### Устраняем ошибки в столбце 'children' 

In [50]:
# еще раз посмотрим на значение в столбце children
df['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

Колличество детей (-1) и 20 ошибочны. Уберем знак минус и ноль.

И информации про 5-х детей маловато....

In [51]:
df.shape

(21424, 12)

In [52]:
# исправим значение с -1 на 1 и с 20 на 2
df['children']=df['children'].replace(-1, 1)

df['children']=df['children'].replace(20, 2)

In [53]:
# у нас всего 9 клиентов в 5 детьми. Очень мало данных. Поэтому их тоже удаляем.
df=df.drop(df[df['children']>4].index)

In [54]:
df.shape

(21415, 12)

Проверили размер датафрейма. Все хорошо. (Т.к. мы удалили "5" (их было 9 ) то и кол-во строк уменьшилось на 9.)

In [55]:
df['children'].value_counts()

0    14080
1     4849
2     2117
3      328
4       41
Name: children, dtype: int64

Проверили результат: кол-во "1" выросло на 46;   кол-во "2" выросло на 76;   "5" исчезли.

Все ок.

### Устраняем ошибки в столбце 'gender' 

In [56]:
# еще раз посмотрим на значения в столбце gender
df['gender'].value_counts()

F      14157
M       7257
XNA        1
Name: gender, dtype: int64

строку со значением XNA удалим

In [57]:
# перед удалением проверим размер таблицы
df.shape

(21415, 12)

In [58]:
# удаляем строку с "XNA"
df=df.drop(df[df['gender']=="XNA"].index)

In [59]:
# после удаления проверим размер таблицы
df.shape

(21414, 12)

In [60]:
# еще раз посмотрим на значения в столбце gender
df['gender'].value_counts()

F    14157
M     7257
Name: gender, dtype: int64

Теперь значения корректны

### Устраняем ошибки в столбце 'income_type' 

удаляем строки, где income_type принимает значение : "предприниматель", "безработный", "студент", "в декрете", т.к. по ним слишком мало данных.

In [61]:
df.shape

(21414, 12)

перед удалением проверяем размер датафрейма

In [62]:
df['income_type'].value_counts()

сотрудник          11058
компаньон           5062
пенсионер           3836
госслужащий         1452
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Вспомним все уникальные значения в нашей колонке

In [63]:
# удаляем выбранные категории
df=df.drop(df[(df['income_type']=='предприниматель')].index)
df=df.drop(df[(df['income_type']=='безработный')].index)
df=df.drop(df[(df['income_type']=='студент')].index)
df=df.drop(df[(df['income_type']=='в декрете')].index)

In [64]:
df.shape

(21408, 12)

после удалением проверяем размер датафрейма

In [65]:
df['income_type'].value_counts()

сотрудник      11058
компаньон       5062
пенсионер       3836
госслужащий     1452
Name: income_type, dtype: int64

на всякий случай смотри и на все уникальные значение в нашей колонке

## 2.2  Замена типа данных

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21408 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21408 non-null  int64  
 1   days_employed     21408 non-null  float64
 2   dob_years         21408 non-null  int64  
 3   education         21408 non-null  object 
 4   education_id      21408 non-null  int64  
 5   family_status     21408 non-null  object 
 6   family_status_id  21408 non-null  int64  
 7   gender            21408 non-null  object 
 8   income_type       21408 non-null  object 
 9   debt              21408 non-null  int64  
 10  total_income      21408 non-null  float64
 11  purpose           21408 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


В датафрейме колонки days_employed и total_income имеют тип данных float, а должен быть int.

In [67]:
df['days_employed']=df['days_employed'].astype('int')
df['total_income']=df['total_income'].astype('int')

переводим столбцы "days_employed" и "total_income" в формат "int". Как учили на спринте!

## 2.3  Обработка дубликатов

In [68]:
df['education'].value_counts()

среднее                13682
высшее                  4683
СРЕДНЕЕ                  769
Среднее                  708
неоконченное высшее      665
ВЫСШЕЕ                   273
Высшее                   265
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 16
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Чтобы избавиться от дубликатов, нужно все привести к нижнему регистру.

In [69]:
df['education']=df['education'].str.lower()

приводим все к нижнему регистру

In [70]:
df['education_id'].value_counts()

1    15159
0     5221
2      741
3      281
4        6
Name: education_id, dtype: int64

Убедились что избавились от дублей в этом столбце. По клиентам имеющим "ученую степень" у нас слишком мало данных. Удаляем их.

In [71]:
df=df[df.education_id<4]

обновим наш датафрейм убрав клиентов с ученой степенью

In [72]:
df.shape

(21402, 12)

проверяем размерность: все верно!

In [73]:
df.duplicated().sum()

71

Обнаружили 71 дубликат! удалим их!

In [74]:
df.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,11342,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21326,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21327,0,11464,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21328,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21329,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


## 2.4  Лемматизация данных

In [75]:
spisok_celei=df['purpose'].unique()
spisok_celei

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

 составляем список всех целей для получения кредита

In [76]:
def funk_celi (spisok,slovo):
    sp=[]
    for i in spisok:
        lemmas=m.lemmatize(i)
        for j in lemmas:
            if j==slovo:
                sp.append(i)
    return sp

функция которая должна разбивать цели для получения кредита на категории

In [77]:
group_obr=funk_celi(spisok_celei, 'образование')
group_gilie=funk_celi(spisok_celei, 'жилье')
group_svadba=funk_celi(spisok_celei, 'свадьба')
group_nedvigimost=funk_celi(spisok_celei, 'недвижимость')
group_avto=funk_celi(spisok_celei, 'автомобиль')

Воспользуемся этой функцией чтобы разбить наш список всех целей по группам

Посмотрим что получилось

In [78]:
group_obr

['дополнительное образование',
 'образование',
 'заняться образованием',
 'получение образования',
 'получение дополнительного образования',
 'получение высшего образования',
 'профильное образование',
 'высшее образование',
 'заняться высшим образованием']

In [79]:
group_gilie

['покупка жилья',
 'операции с жильем',
 'покупка жилья для семьи',
 'жилье',
 'покупка своего жилья',
 'покупка жилья для сдачи',
 'ремонт жилью']

In [80]:
group_svadba

['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']

In [81]:
group_nedvigimost

['покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'операции со своей недвижимостью',
 'операции с недвижимостью']

In [82]:
group_avto

['приобретение автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'автомобили',
 'сделка с подержанным автомобилем',
 'автомобиль',
 'свой автомобиль',
 'сделка с автомобилем',
 'на покупку автомобиля']

 Список целей "разбился" по категориям.

Лемматизация сделана по принципу разбияния на группы по смыслу. Так, для каждой подкатегории выбрано ключевое слово объединяющее "цели получения кредита". Для группы - недвижимость, объединили леммы "жилье" и "недвижимость", так как по сути это синонимы.

## 2.5  Категоризация данных

### Категоризируем данные по "цели кредита"

In [83]:
def funk_id(group):
    if group in group_svadba:
        return 'cvadba'
    
    elif group in group_avto:
            return 'avto'
        
    elif group in group_nedvigimost:
            return 'nedvigimost'
        
    elif group in group_gilie:
            return 'nedvigimost'
        
    elif group in group_obr:
            return 'obrazovanie'
    return group

Функция для разбиения на подкатегории: cvadba(свадьба), avto(автомобиль), nedvigimost(недвидимость), obrazovanie(образование).

Объединим понятия "недвижимость" и "жилье", т.к. это практически одно и то же.


In [84]:
df['purpose_id']=df['purpose'].apply(funk_id)

применяем нашу функцию "funk_id"

In [85]:
df['purpose_id'].value_counts()

nedvigimost    10778
avto            4290
obrazovanie     4002
cvadba          2332
Name: purpose_id, dtype: int64

в основном клиенты в данной выборке берут кредиты на недвижимость

### Категоризируем колонки "уровень дохода"

In [86]:
def uroven_dohoda(kolonka):
    if kolonka<30000:
        return 'низкий уровень'
    elif kolonka<100000:
        return 'средний уровень'
    elif kolonka <180000:
        return 'высокий уровень'
    else:
        return 'очень высокий уровень'

Напишем функцию для группировки уровней дохода. Разобьем на 4 группы: 'низкий уровень','средний уровень', 'высокий уровень', 'очень высокий уровень'.

In [87]:
df['uroven_dohoda']=df['total_income'].apply(uroven_dohoda)

Применяем функцию к столбцу "total_income".

In [88]:
df['uroven_dohoda'].value_counts()

высокий уровень          10401
очень высокий уровень     6567
средний уровень           4412
низкий уровень              22
Name: uroven_dohoda, dtype: int64

Большинство клиентов из нашей выборки с высоким уровнем доходов.

Принцип выбора категорий:

Самое логичное было бы погуглить официальную статистику по РФ (т.к. регион проживания нам не дан) и посмотреть сколько люди зарабатывают, но к сожалению, при таком подходе разбивка получается сильно разбалансированной: огромное кол-во людей с высоким и очень высоким уровнем доходов, т.к. официальные цифры весьма скромные. Множество других исследований (банков/экономистов/журналистов и т.д.) субьективно и может сильно различаться. 
    
Поэтому я взял цифры ориентируясь на собственный опыт - зарплаты специалистов относительно должностей и стажа работы. Низкий уровень = молодой специалист без опыта работы, средний уровень = опыт работы более 3-х - 5 лет, высокий уровень = начальник отдела/отделения, очень высокий уровень = высокое начальство/топ менеджеры. 

### Категоризируем колонку трудовой стаж

In [89]:
def uroven_trudovogo_staga(kolonka):
    if kolonka<1095:
        return 'меньше 3 лет'
    elif kolonka<3650:
        return 'от 3-х до 10 лет'
    else:
        return 'от 10 лет'

Напишем функцию для группировки трудового стажа. Разобьем на 3 группы: 'меньше 3 лет','от 3-х до 10 лет', 'от 10 лет'.

1095 - кол-во дней в 3-х годах

3650 - кол-во дней в 10 годах

In [90]:
# применяем функцию к датафрейму
df['days_employed_id']=df['days_employed'].apply(uroven_trudovogo_staga)
df['days_employed_id'].value_counts()

от 3-х до 10 лет    8992
от 10 лет           6558
меньше 3 лет        5852
Name: days_employed_id, dtype: int64

Большинство клиентов из нашей выборки имеют трудовой стаж от 3-х до 10 лет.

### Категоризируем колонку "возраст"

In [91]:
def uroven_age(kolonka):
    if kolonka<23:
        return 'возраст выпускника вуза(до 23)'
    elif kolonka<35:
        return 'возраст условно "молодого человека"(23-35)'
    elif kolonka<65:
        return 'возраст зрелого человека(35-65)'
    else:
        return 'возраст пожилого человека (пенсионера)(от 65)'

Напишем функцию для группировки по возрасту. Разобьем на 3 группы: 'меньше 3 лет','от 3-х до 10 лет', 'от 10 лет'.

In [92]:
# применяем функцию к датафрейму
df['dob_years_id']=df['dob_years'].apply(uroven_age)

# смотрим на результат
df['dob_years_id'].value_counts()

возраст зрелого человека(35-65)                  14532
возраст условно "молодого человека"(23-35)        5614
возраст пожилого человека (пенсионера)(от 65)      898
возраст выпускника вуза(до 23)                     358
Name: dob_years_id, dtype: int64

Большинство клиентов из нашей выборки зрелого возраста (35лет-65лет).

Группа людей до 23 лет - это группа молодых людей , большое кол-во студентов. Группа людей от 23 до 35 лет - это группа людей так же молодых (по закону молодым человек считается до 35 лет), так же подавляющее большинство людей в этой восрастной группе уже не учится, а работает. Группа от 35-65 лет - это группа людей среднего возраста, имеющих устоявшийся образ жизни и большой стаж работы (как правило). Группа от 65 лет - это пенсионеры (взял именно 65, а не 60, т.к. после 65 человек ТОЧНО пенсионер (60/65 = пенсионный возраст женщин/мужчин)).

# 3. Анализ данных

### Анализируем наличие задолжностей по колличеству детей

Для все приведенных ниже результатов мы будем делить кол-во должников на общее кол-во (по каждой категори).

In [93]:
table_children_sum=df.groupby('children', as_index=False).agg({'debt':"sum"})
table_children_sum['count']=df.groupby('children', as_index=False).agg({'debt':"count"})['debt']
table_children_sum['res']=table_children_sum['debt']/table_children_sum['count']*100
table_children_sum

,children,debt,count,res
0,0,1058,14071,7.519011
1,1,441,4847,9.098411
2,2,201,2116,9.499055
3,3,27,327,8.256881
4,4,4,41,9.756098


Вывод: Клиенты не имеющие детей чаще возвращают платежи в срок. Клиенты у которых 4 детей хуже всех возвращают платежи (хотя по ним не так уж и много данных - всего 41 случай).

### Анализируем наличие задолжностей по семейному статусу

In [94]:
table_family_status_sum=df.groupby('family_status', as_index=False).agg({'debt':"sum"})
table_family_status_sum['count']=df.groupby('family_status', as_index=False).agg({'debt':"count"})['debt']
table_family_status_sum['res']=table_family_status_sum['debt']/table_family_status_sum['count']*100
table_family_status_sum

,family_status,debt,count,res
0,Не женат / не замужем,273,2795,9.767442
1,в разводе,85,1184,7.179054
2,вдовец / вдова,62,955,6.492147
3,гражданский брак,386,4151,9.298964
4,женат / замужем,925,12317,7.509946


Вывод: категории: "вдовец / вдова" и в "разводе" чаще возвращают платеж в срок. Хуже всех - клиенты в статусе "не женат/не замужем"

### Анализируем наличие задолжностей по уровню дохода

In [95]:
table_uroven_dohoda_sum=df.groupby('uroven_dohoda', as_index=False).agg({'debt':"sum"})
table_uroven_dohoda_sum['count']=df.groupby('uroven_dohoda', as_index=False).agg({'debt':"count"})['debt']
table_uroven_dohoda_sum['res']=table_uroven_dohoda_sum['debt']/table_uroven_dohoda_sum['count']*100
table_uroven_dohoda_sum

,uroven_dohoda,debt,count,res
0,высокий уровень,896,10401,8.614556
1,низкий уровень,2,22,9.090909
2,очень высокий уровень,484,6567,7.370184
3,средний уровень,349,4412,7.910245


Вывод: Люди с "очень высоким уровнем" дохода реже "забывают" внести платеж. Люди с низким уровнем дохода чаще остальных задерживают платежи.

### Анализируем наличие задолжностей по уровню образования

In [96]:
table_education_sum=df.groupby('education', as_index=False).agg({'debt':"sum"})
table_education_sum['count']=df.groupby('education', as_index=False).agg({'debt':"count"})['debt']
table_education_sum['res']=table_education_sum['debt']/table_education_sum['count']*100
table_education_sum

,education,debt,count,res
0,высшее,277,5221,5.305497
1,начальное,31,281,11.032028
2,неоконченное высшее,68,741,9.176788
3,среднее,1355,15159,8.938584


Вывод: Клиенты с высшим образованием лучше остальных вносят платежи. Чаще всех пропускают платежи клиенты с начальным образованием. 

### Анализируем наличие задолжностей по цели получения кредита

In [97]:
table_purpose_id_sum=df.groupby('purpose_id', as_index=False).agg({'debt':"sum"})
table_purpose_id_sum['count']=df.groupby('purpose_id', as_index=False).agg({'debt':"count"})['debt']
table_purpose_id_sum['res']=table_purpose_id_sum['debt']/table_purpose_id_sum['count']*100
table_purpose_id_sum

,purpose_id,debt,count,res
0,avto,399,4290,9.300699
1,cvadba,184,2332,7.890223
2,nedvigimost,778,10778,7.218408
3,obrazovanie,370,4002,9.245377


Вывод: Клиенты, целью которых, является операции с недвижимостью реже пропускают платежи. Клиенты, целью которых, является автомобиль (или образование) чаще пропускают платежи.

### Анализируем наличие задолжностей по гендорному признаку

In [98]:
table_gender_sum=df.groupby('gender', as_index=False).agg({'debt':"sum"})
table_gender_sum['count']=df.groupby('gender', as_index=False).agg({'debt':"count"})['debt']
table_gender_sum['res']=table_gender_sum['debt']/table_gender_sum['count']*100
table_gender_sum

,gender,debt,count,res
0,F,988,14152,6.981345
1,M,743,7250,10.248276


Вывод: Женщины реже "забывают внести" платеж,чем мужчины

### Анализируем наличие задолжностей по типу занятости

In [99]:
table_income_type_sum=df.groupby('income_type', as_index=False).agg({'debt':"sum"})
table_income_type_sum['count']=df.groupby('income_type', as_index=False).agg({'debt':"count"})['debt']
table_income_type_sum['res']=table_income_type_sum['debt']/table_income_type_sum['count']*100
table_income_type_sum

,income_type,debt,count,res
0,госслужащий,86,1451,5.926947
1,компаньон,375,5062,7.408139
2,пенсионер,215,3834,5.607720
3,сотрудник,1055,11055,9.543193


Вывод: Лучше всех вносят оплату клиенты "пенсионеры". Хуже всех вносят оплату клиенты "сотрудники". По группам клиентов: безработный, в декрете, предприниматель, студент - мало данных.

### Анализируем наличие задолжностей по трудовому стажу

In [100]:
table_days_employed_id_sum=df.groupby('days_employed_id', as_index=False).agg({'debt':"sum"})
table_days_employed_id_sum['count']=df.groupby('days_employed_id', as_index=False).agg({'debt':"count"})['debt']
table_days_employed_id_sum['res']=table_days_employed_id_sum['debt']/table_days_employed_id_sum['count']*100
table_days_employed_id_sum

,days_employed_id,debt,count,res
0,меньше 3 лет,642,5852,10.970608
1,от 10 лет,342,6558,5.215005
2,от 3-х до 10 лет,747,8992,8.307384


Вывод: клиенты с наилучшими показателями имеют трудовой стаж более 10 лет. Клиенты с плохими показателями имеют трудовой стаж меньше 3 лет.

### Анализируем наличие задолжностей по категории возраста

In [101]:
table_dob_years_id_sum=df.groupby('dob_years_id', as_index=False).agg({'debt':"sum"})
table_dob_years_id_sum['count']=df.groupby('dob_years_id', as_index=False).agg({'debt':"count"})['debt']
table_dob_years_id_sum['res']=table_dob_years_id_sum['debt']/table_dob_years_id_sum['count']*100
table_dob_years_id_sum

,dob_years_id,debt,count,res
0,возраст выпускника вуза(до 23),44,358,12.290503
1,возраст зрелого человека(35-65),1032,14532,7.101569
2,возраст пожилого человека (пенсионера)(от 65),49,898,5.456570
3,"возраст условно ""молодого человека""(23-35)",606,5614,10.794442


Вывод: Клиент старше 65 лет является самым хорошим, с точки зрения задолжностей по выплатам. Молодые люди до 23 лет довольно проблемны с точки зрения задолжностей по выплатам.

In [102]:
table_dob_years_id_pivot=df.pivot_table(index=['dob_years_id'], values='debt', aggfunc='sum')
table_dob_years_id_pivot['count']=df.pivot_table(index=['dob_years_id'], values='debt', aggfunc='count')
table_dob_years_id_pivot['res']=table_dob_years_id_pivot['debt']/table_dob_years_id_pivot['count']*100
table_dob_years_id_pivot

,debt,count,res
dob_years_id,,,
возраст выпускника вуза(до 23),44,358,12.290503
возраст зрелого человека(35-65),1032,14532,7.101569
возраст пожилого человека (пенсионера)(от 65),49,898,5.456570
"возраст условно ""молодого человека""(23-35)",606,5614,10.794442


# 4. ВЫВОД

Проанализировав датафрейм, характерными особенностями у БОЛЬШИНСТВА клиентов которого являлось:
- кол-во детей=0,
- уровень образования=среднее,
- семейный статус=женат/замужем,
- пол=женщины,
- тип занятости="сотрудник",
- цель получения кредита="недвижимость",
- уровень дохода="высокий"(от 100т.р. - 180т.р.),
- трудовой стаж="от 3-х до 10 лет",
- возрастом= 35лет-65лет

Из-за недостатка данных в процессе работы мы отсеяли колонки "children"(кол-во: 5 детей), "income_type" ("предприниматель", "безработный", "студент", "в декрете"), "education"("ученая степень").

Признаки самого лучшего клиента, с точки зрения "задолженностей":
- Пол: женщина,
- Кол-во детей: 0,
- Семейное положение: вдова/вдовец,
- Уровень дохода: очень высокий(от 100т.р. до 180т.р.),
- Уровень образования: высшее образование,
- Цель получения кредита: операции с недвижимостью,
- Тип занятости: пенсионер,
- Трудовой стаж: от 10 лет,
- Категория возраста: старше 65 лет.


Признаки самого проблемного клиента, с точки зрения "задолженностей":
- Пол: мужчина,
- Кол-во детей: >1,
- Семейное положение: не женат/не замужем,
- Уровень дохода: очень низкий( до 30т.р.),
- Уровень образования: начальное образование,
- Цель получения кредита: автомобиль (или образование),
- Тип занятости: сотрудник,
- Трудовой стаж: до 3 лет,
- Категория возраста: до 23 лет.

Рекомендации.
Наиболее "сильные" зависимости данных были у признаков: пол, уровень образования, трудовой стаж, возраст.